# Time Series Analysis of HiPSCats
Here we will use the lsdb framework to load ztf objects and sources then perform quick computations obtain some lightcurve information. 

We will do the same joint analysis as before, where we will find a subset of high-proper motion stars with gaia, that have a lot of observations in ztf. Here, we will join the result to the ztf source catalog, and extract lightcurve information for our subset.

In [1]:
#Start up our distributed client
from lsd2 import lsd2_client
client = lsd2_client(dask_on_ray=True, num_workers=12)

2023-07-13 16:54:53,898	INFO worker.py:1636 -- Started a local Ray instance.
/astro/users/sdwyatt/anaconda3/envs/lsdb-test/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(


#### Here we gather our datasets that have already been partitioned in the hipscat manner

In [2]:
import lsdb
import numpy as np
gaia = lsdb.read_hipscat("/data3/epyc/projects3/ivoa_demo/gaia/catalog")
ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")

#### Cull the gaia through conesearch and proper motion calculation

In [5]:
result = gaia.cone_search(
    ra=30,
    dec=30,
    radius=1,
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).query(
    "pm > 100"
)

result.compute()

,solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,Norder,Npix,Dir
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
599985615328509952,1636148068921376768,Gaia DR3 299992882826804608,299992882826804608,806471176,2016.0,30.752383,0.232542,29.286089,0.249211,2.086858,...,0.0117,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33
600006703617933312,1636148068921376768,Gaia DR3 300003435561180800,300003435561180800,1457731509,2016.0,30.503461,0.667044,29.332372,0.703648,8.704457,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
600044001113931776,1636148068921376768,Gaia DR3 300022058539824256,300022058539824256,1362016597,2016.0,30.848141,0.046323,29.471117,0.047651,3.538279,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
600078064499556352,1636148068921376768,Gaia DR3 300039337192927360,300039337192927360,455703316,2016.0,30.683537,0.016001,29.610079,0.013207,7.750687,...,0.0066,0.0022,0.0127,0.0035,0.0012,0.0067,MARCS,3,33,33
600102159266086912,1636148068921376768,Gaia DR3 300051260022544512,300051260022544512,213865764,2016.0,30.914979,0.873999,29.892982,0.851259,10.365376,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
600853615334129664,1636148068921376768,Gaia DR3 300426988056384128,300426988056384128,72968762,2016.0,31.049614,0.055481,29.984640,0.058904,3.303467,...,0.2879,0.2457,0.4299,0.1558,0.1332,0.2328,PHOENIX,3,33,33
601440514025193472,1636148068921376768,Gaia DR3 300720347207348864,300720347207348864,1664182609,2016.0,29.169068,0.117511,29.594295,0.116793,10.827253,...,NaN,NaN,NaN,NaN,NaN,NaN,None,3,33,33
601459416176263168,1636148068921376768,Gaia DR3 300729800430796160,300729800430796160,1337323927,2016.0,29.732362,0.012473,29.530035,0.013096,11.425982,...,0.1561,0.1504,0.1605,0.0842,0.0811,0.0866,MARCS,3,33,33
601519129106579456,1636148068921376768,Gaia DR3 300759624683681280,300759624683681280,1506362931,2016.0,30.268854,0.026717,29.477900,0.027353,1.221485,...,0.1685,0.1613,0.1722,0.0923,0.0883,0.0944,MARCS,3,33,33


#### Cross match our result with ztf

In [6]:
xmatch = ztf.crossmatch(result)
xmatch.compute()

,ps1_objid_ztf_dr14,ra_ztf_dr14,dec_ztf_dr14,ps1_gMeanPSFMag_ztf_dr14,ps1_rMeanPSFMag_ztf_dr14,ps1_iMeanPSFMag_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,...,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,Norder_gaia,Npix_gaia,Dir_gaia,_DIST
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
599985615462727680,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33,0.009096
599985619128549376,143140307414235473,30.741507,29.287432,21.074301,19.833200,18.597900,227,622,43,21.078733,...,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33,0.009581
599985683452395520,143140307622352925,30.762255,29.285293,21.867001,20.597500,19.726299,41,418,38,21.576769,...,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33,0.008648
599985694294671360,143140307622677253,30.762289,29.288910,16.239300,15.852000,15.715900,560,710,44,16.263656,...,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33,0.009088
599985795469672448,143150307536922778,30.753726,29.293489,-999.000000,21.714500,20.282101,0,97,29,NaN,...,0.0028,0.0252,0.0063,0.0015,0.0136,PHOENIX,3,33,33,0.007493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602923217291575296,144990302509326955,30.250933,30.830336,18.956100,17.956600,17.520201,484,717,55,19.038398,...,0.5523,0.5670,0.3603,0.3555,0.3646,MARCS,3,33,33,0.005747
602923228884631552,145000302522403797,30.252492,30.835897,17.536301,16.329800,14.985600,498,727,57,17.632277,...,0.5523,0.5670,0.3603,0.3555,0.3646,MARCS,3,33,33,0.000105
602923234492416000,145000302561593551,30.256145,30.835851,16.508699,16.083799,15.927400,501,726,57,16.540697,...,0.5523,0.5670,0.3603,0.3555,0.3646,MARCS,3,33,33,0.003231


#### Next, join with ztf sources on their ps1_objectid

In [7]:
join = xmatch.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
).compute()

#### Filter that result based on many observations to get our lightcurve data

In [8]:
join.query(
    "nobs_g_ztf_dr14 > 20 and nobs_r_ztf_dr14 > 20 and nobs_i_ztf_dr14 > 20"
)

,ps1_objid_ztf_dr14,ra_ztf_dr14,dec_ztf_dr14,ps1_gMeanPSFMag_ztf_dr14,ps1_rMeanPSFMag_ztf_dr14,ps1_iMeanPSFMag_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,...,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
0,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0,604,19.866035,0.107689,59800.45349,56,r,7,0,8526
1,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0,604,19.716253,0.096444,59796.45919,56,r,7,0,8526
2,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0,604,19.736191,0.097881,59787.42878,56,r,7,0,8526
3,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0,604,19.975979,0.116594,59778.45889,56,r,7,0,8526
4,143140307421131702,30.742162,29.284280,20.062799,19.783001,19.688299,449,593,37,20.068068,...,0,604,19.982220,0.117115,59778.42886,56,r,7,0,8526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5629,145010302517433949,30.251740,30.844494,16.517200,16.194500,16.058500,501,726,57,16.544471,...,0,652,16.547066,0.014516,58514.17352,9,g,7,0,8568
5630,145010302517433949,30.251740,30.844494,16.517200,16.194500,16.058500,501,726,57,16.544471,...,0,652,16.551884,0.014533,58511.14248,9,g,7,0,8568
5631,145010302517433949,30.251740,30.844494,16.517200,16.194500,16.058500,501,726,57,16.544471,...,0,652,16.546101,0.014512,58487.18088,9,g,7,0,8568
5632,145010302517433949,30.251740,30.844494,16.517200,16.194500,16.058500,501,726,57,16.544471,...,-32768,652,16.531082,0.014460,58488.16219,9,g,7,0,8568


In [9]:
client.shutdown()

/astro/users/sdwyatt/anaconda3/envs/lsdb-test/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(
